In [1]:
import slicer, vtk
import numpy as np
import vtk.util.numpy_support
import numpy as np
import pandas as pd
from my_functions import *
import gc
import os
import sys
sys.path.append('../')
from config import load_config

In [2]:
config = load_config(os.path.abspath("../config.yml"))
df_path = config.DATA_FILE
mrb_save_dir = config.MRB_SAVE_PATH
numpy_save_dir = config.NUMPY_SAVE_PATH
os.makedirs(os.path.join(numpy_save_dir), exist_ok=True)

In [3]:
def convertToEQD2(dose, fraction):
    alpha_beta_ratio = 3
    eqd2 = dose * (dose/fraction + alpha_beta_ratio)/(2 + alpha_beta_ratio)
    return eqd2


def resizeVolume(newSpacing, originalNode, newNode, method="linear"):
    parameters = {}
    parameters["outputPixelSpacing"] = newSpacing
    parameters["interpolationType"] = method

    ##d = slicer.cli.createNode(slicer.modules.resamplescalarvolume, parameters=None)
    ##d.GetParameterName(0,1)
    parameters["InputVolume"] = originalNode
    parameters["OutputVolume"] = newNode

    # Execute
    resampleVolume = slicer.modules.resamplescalarvolume
    cliNode = slicer.cli.runSync(resampleVolume, None, parameters)

    if cliNode.GetStatus() & cliNode.ErrorsMask:
        # error
        errorText = cliNode.GetErrorText()
        slicer.mrmlScene.RemoveNode(cliNode)
        raise ValueError("CLI execution failed: " + errorText)
        
    return newNode

In [4]:
def calculateDoseMapAndSave(patient, prescription, frac, ptvs, igtvs):
    new_volume_dims = (3,3,3)
    num_lesions = len(ptvs)

    prescription_list = prescription
    frac_list = frac
    
    #Convert prescription to EQD2
    for i in range(len(prescription_list)):
        prescription_list[i] = convertToEQD2(float(prescription_list[i]), int(frac_list[i]))
    
    
    #These are final coefficients from 54 single lesion plans
    A1 = 91.9
    a1 = 0.104
    A2 = 8.09
    a2 = 0.027
    slicer.mrmlScene.Clear(0)

    try:
        slicer.util.loadScene(os.path.join(mrb_save_dir, patient + ".mrb"))
    except:
        pass

    volumeNode = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[0]
    
    segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
    segmentation = segmentationNode.GetSegmentation()

    #Getting actual dose map
    for i in range(len(slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode"))):
        origDoseNode  = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")[i]
        if "dose" in origDoseNode.GetName().lower():
            break
    
    ptvs_from_names = []
    for ptv in ptvs:
        ptvs_from_names.append(segmentation.GetSegmentIdBySegmentName(ptv))

    #Getting dose maps for all lesions  
    Mm = []    
    
    for i in range(num_lesions):
        Mm.append(creatingCustomDoseMap(volumeNode, float(prescription_list[i]), coeffs=[A1,a1,A2,a2], ptvID=ptvs_from_names[i]))
        
    referenceShape = Mm[0].shape
        
    Mm_summed = np.zeros(referenceShape)
    
    for i in range(num_lesions):
        Mm_summed = Mm_summed + Mm[i]
    
    MmNode = createVolumeNode(Mm_summed, volumeNode, "ExponentialEstimate")
    convertToRTNode(MmNode, volumeNode)

    resampledOriginalDoseNode = resampleScalarVolumeBrains(origDoseNode, volumeNode)
    
    #Converting to RT dose node
    convertToRTNode(resampledOriginalDoseNode, volumeNode)
    ogDoseArr = slicer.util.arrayFromVolume(resampledOriginalDoseNode)
    
    IDs = vtk.vtkStringArray()
    segmentation.GetSegmentIDs(IDs)

    
    
    oar_dict = {"Lung": 1,
                "Heart": 2,
                "Eso": 3,
                "CW": 4,
                "GV": 2,
                "Trachea": 5,
                "BT": 5, 
                "External": -1,
                "SpinalCanal": 6,
                }
    

    lung_R_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.LUNG_RIGHT), referenceShape) * oar_dict["Lung"]
    lung_L_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.LUNG_LEFT), referenceShape) * oar_dict["Lung"]
    heart_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.HEART), referenceShape) * oar_dict["Heart"]
    esophagus_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.ESOPHAGUS), referenceShape) * oar_dict["Eso"]
    chestWall_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.CHESTWALL), referenceShape) * oar_dict["CW"]
    greatVessels_array = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName("GV_Combined"), referenceShape) * oar_dict["GV"]
    tracheaArray = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.TRACHEA), referenceShape) * oar_dict["Trachea"]
    bronchialTreeArray = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.PULMONARYBRONCHIALTREE), referenceShape) * oar_dict["BT"]
    externalArray = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.BODY), referenceShape) * oar_dict["External"]
    spinalCanalArray = getFullSizeSegmentation(segmentation.GetSegmentIdBySegmentName(config.SPINALCANAL), referenceShape) * oar_dict["SpinalCanal"]
    
    
    ptv_arrays = []
    for i in range(num_lesions):
        ptv_arrays.append(getFullSizeSegmentation(ptvs_from_names[i], referenceShape) * (10 + i))
    ptv_array = np.sum(ptv_arrays,axis=0)
    
    del ptv_arrays
    gc.collect()

    #Build OARs array, accounting from overlap
    oars = externalArray.copy()
    oars[np.where(spinalCanalArray>0)] = oar_dict["SpinalCanal"]
    oars[np.where(heart_array>0)] = oar_dict["Heart"]
    oars[np.where(greatVessels_array>0)] = oar_dict["GV"]
    oars[np.where(esophagus_array>0)] = oar_dict["Eso"]
    oars[np.where(bronchialTreeArray>0)] = oar_dict["BT"]
    oars[np.where(tracheaArray>0)] = oar_dict["Trachea"]
    oars[np.where(chestWall_array>0)] = oar_dict["CW"]
    oars[np.where(lung_R_array>0)] = oar_dict["Lung"]
    oars[np.where(lung_L_array>0)] = oar_dict["Lung"]
    
    oars[np.where(ptv_array>0)] = 10

    oarNode = createVolumeNode(oars, volumeNode, "OARs")
    
    combined_lung_array = np.sum([lung_R_array, lung_L_array],axis=0)
    
    lungNode = createVolumeNode(combined_lung_array, volumeNode, "Lungs")
    ptvNode = createVolumeNode(ptv_array, volumeNode, "PTVs")

    #resize volume node to 3x3x3
    
    ogDoseArrResizedNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedOgDose")
    Mm_summedResizedNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedMm")
    oarNodeResizedNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedOAR")
    lungResizedNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedLung")
    ptvResizedNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedPTVs")

    ogDoseArrResizedNode = resizeVolume(new_volume_dims, resampledOriginalDoseNode, ogDoseArrResizedNode)
    Mm_summedResizedNode = resizeVolume(new_volume_dims, MmNode, Mm_summedResizedNode)
    oarNodeResizedNode = resizeVolume(new_volume_dims, oarNode, oarNodeResizedNode, "nearestNeighbor")
    lungResizedNode = resizeVolume(new_volume_dims, lungNode, lungResizedNode, "nearestNeighbor")    
    ptvResizedNode = resizeVolume(new_volume_dims, ptvNode, ptvResizedNode, "nearestNeighbor")
    
    ##OARs with IGTV
    igtvs_from_names = []
    for igtv in igtvs:
        igtvs_from_names.append(segmentation.GetSegmentIdBySegmentName(igtv))
    
    igtv_arrays = []
    for i in range(num_lesions):
        igtv_arrays.append(getFullSizeSegmentation(igtvs_from_names[i], referenceShape))
    igtv_array = np.sum(igtv_arrays,axis=0)
    
    del igtv_arrays
    gc.collect()

    oarsIgtv = externalArray.copy()
    oarsIgtv[np.where(spinalCanalArray>0)] = oar_dict["SpinalCanal"]
    oarsIgtv[np.where(heart_array>0)] = oar_dict["Heart"]
    oarsIgtv[np.where(greatVessels_array>0)] = oar_dict["GV"]
    oarsIgtv[np.where(esophagus_array>0)] = oar_dict["Eso"]
    oarsIgtv[np.where(bronchialTreeArray>0)] = oar_dict["BT"]
    oarsIgtv[np.where(tracheaArray>0)] = oar_dict["Trachea"]
    oarsIgtv[np.where(chestWall_array>0)] = oar_dict["CW"]
    oarsIgtv[np.where(lung_R_array>0)] = oar_dict["Lung"]
    oarsIgtv[np.where(lung_L_array>0)] = oar_dict["Lung"]
    oarsIgtv[np.where(igtv_array>0)] = 10
    
    #Deleting variables to free up memory
    del spinalCanalArray
    del heart_array
    del greatVessels_array
    del esophagus_array
    del bronchialTreeArray
    del tracheaArray
    del chestWall_array
    del lung_R_array
    del lung_L_array
    gc.collect()
    
    oarIGTVNode = createVolumeNode(oarsIgtv, volumeNode, "OARsIGTV")
    oarNodeResizedIGTVNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedIGTV") 
    oarNodeResizedIGTVNode = resizeVolume(new_volume_dims, oarIGTVNode, oarNodeResizedIGTVNode, "nearestNeighbor")
    
    CTNode = createVolumeNode(slicer.util.arrayFromVolume(volumeNode) , volumeNode, "CT")
    resizedCTNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "ResizedCT") 
    resizedCTNode = resizeVolume(new_volume_dims, CTNode, resizedCTNode)
        

    ogDose333 = slicer.util.arrayFromVolume(ogDoseArrResizedNode) 
    Mm333 = slicer.util.arrayFromVolume(Mm_summedResizedNode) 
    oar333 = slicer.util.arrayFromVolume(oarNodeResizedNode) 
    lung333 = slicer.util.arrayFromVolume(lungResizedNode) 
    oarigtv333 = slicer.util.arrayFromVolume(oarNodeResizedIGTVNode)
    ct333 = slicer.util.arrayFromVolume(resizedCTNode)
    ptv333 = slicer.util.arrayFromVolume(ptvResizedNode)
    
    
    #Clip ct333 to be between -1024 and 3071
    ct333 = np.clip(ct333, -1024, 3071)
    

    not_zero_inds = np.where(lung333!=0)
    min_x, min_y, min_z = np.min(not_zero_inds[0]), np.min(not_zero_inds[1]), np.min(not_zero_inds[2])
    max_x, max_y, max_z = np.max(not_zero_inds[0]), np.max(not_zero_inds[1]), np.max(not_zero_inds[2])
    center_x = int((min_x + max_x)/2)
    center_y = int((min_y + max_y)/2)
    center_z = int((min_z + max_z)/2)
    

    new_x_min = center_x - 42
    new_x_max = center_x + 42
    
    new_y_min = center_y - 72
    new_y_max = center_y + 72
    
    new_z_min = center_z - 72
    new_z_max = center_z + 72
    
    if new_x_min < 0:
        adj = -new_x_min
        new_x_min += adj
        new_x_max += adj
    if new_y_min < 0:
        adj = -new_y_min
        new_y_min += adj
        new_y_max += adj
    if new_z_min < 0:
        adj = -new_z_min
        new_z_min += adj
        new_z_max += adj
    if new_x_max >= lung333.shape[0]:
        adj = new_x_max - lung333.shape[0]
        new_x_min -= adj
        new_x_max -= adj
    if new_y_max >= lung333.shape[1]:
        adj = new_y_max - lung333.shape[1]
        new_y_min -= adj
        new_y_max -= adj
    if new_z_max >= lung333.shape[2]:
        adj = new_z_max - lung333.shape[2]
        new_z_min -= adj
        new_z_max -= adj
    
    ogDoseFinal = ogDose333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    MmFinal = Mm333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    oarFinal = oar333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    ctFinal = ct333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    ptvFinal = ptv333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    
    #Making a new array for raw doses
    doseOnlyFinal = np.zeros(ogDoseFinal.shape)
    for i in range(num_lesions):
        doseOnlyFinal = doseOnlyFinal + (ptvFinal == (10 + i)) * prescription_list[i] 
    
    stack = np.stack([ogDoseFinal,MmFinal,oarFinal, ctFinal, doseOnlyFinal])

    assert stack[0].shape == stack[1].shape == stack[2].shape
    
    oarIgtvFinal = oarigtv333[new_x_min:new_x_max, new_y_min:new_y_max, new_z_min:new_z_max]
    stackIgtv = np.stack([ogDoseFinal,MmFinal,oarIgtvFinal, ctFinal, doseOnlyFinal])
    
    
    assert len(np.unique(oarFinal)) == 9
    assert len(np.unique(oarIgtvFinal)) == 9
    
    stack_both = np.stack([ogDoseFinal, MmFinal,oarFinal, ctFinal, doseOnlyFinal, oarIgtvFinal])

    return stack, stackIgtv, stack_both

In [5]:
df = pd.read_csv(df_path)

for j in range(len(df)):
    PTVs = df.iloc[j]["PTVs"].split(",")
    IGTVs = df.iloc[j]["IGTVs"].split(",")
    if len(PTVs) > 1:
        doses = df.iloc[j]["Dose"].split(",")
        fractions = df.iloc[j]["Fraction"].split(",")
    else:
        doses = [df.iloc[j]["Dose"]]
        fractions = [df.iloc[j]["Fraction"]]
    patient = df.iloc[j]["Patient"]
    stack, stackIgtv, stack_both = calculateDoseMapAndSave(patient, doses, fractions, PTVs, IGTVs)
    np.save(os.path.join(numpy_save_dir, patient + ".npy"), stack_both)

print("end")

end
